In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

cifar10 = keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

train_labels = keras.utils.to_categorical(train_labels, 10)
test_labels = keras.utils.to_categorical(test_labels, 10)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
import time
import math

#Set equal to 1 for SGD and 2 for Adam optimizer
opt_selection = 1

def lr_schedule(epoch):
    lr_initial = 0.1
    drop = 0.5
    drop_interval = 25.0
    lr = lr_initial * math.pow(drop, math.floor((1 + epoch)/drop_interval))
    return lr
lrate = LearningRateScheduler(lr_schedule)

weight_decay = 1e-4

model = Sequential()
model.add(Conv2D(32, (3, 3), kernel_regularizer=l2(weight_decay), padding = 'same', input_shape = train_images.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), kernel_regularizer=l2(weight_decay), padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), kernel_regularizer=l2(weight_decay), padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), kernel_regularizer=l2(weight_decay), padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), kernel_regularizer=l2(weight_decay), padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), kernel_regularizer=l2(weight_decay), padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(10, activation = 'softmax'))

train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

mean = np.mean(train_images,axis=(0,1,2,3))
std = np.std(train_images,axis=(0,1,2,3))
train_images = (train_images-mean)/(std)
test_images = (test_images-mean)/(std)

datagen = ImageDataGenerator(
    featurewise_center = False,
    featurewise_std_normalization = False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True)
datagen.fit(train_images)

if opt_selection == 1:
    opt = SGD(lr = 0.1, momentum = 0.9, nesterov = True)
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model.summary()
    
    start = time.time()
    data = model.fit_generator(datagen.flow(train_images, train_labels, batch_size = 128),
                    validation_data=(test_images, test_labels),
                    steps_per_epoch = np.ceil(len(train_images) / 128),
                    epochs = 100, verbose=2, callbacks = [lrate])
elif opt_selection == 2:
    opt = Adam(lr = 0.01)
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model.summary()
    
    start = time.time()
    data = model.fit_generator(datagen.flow(train_images, train_labels, batch_size = 128),
                    validation_data=(test_images, test_labels),
                    steps_per_epoch = np.ceil(len(train_images) / 128),
                    epochs = 100, verbose=2)

end = time.time()

In [ ]:
import matplotlib.pyplot as plt
print("Total training time: ", end - start, " seconds.")

plt.plot(data.history['acc'])
plt.plot(data.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()

plt.plot(data.history['loss'])
plt.plot(data.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()

In [ ]:
import json
file_name = "ModelResults_CIFAR-10.json"
with open(file_name, 'w') as file:
    json.dump(str(data.history), file)